In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
OUTPUT_FOLDER = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/RGB_feature/labelled_images"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
# Loading stored reference features from CSV
def load_reference_features(csv_file):
    return pd.read_csv(csv_file, header=None).values.flatten()

In [ ]:
# Function to extract texture and color features
def extract_features(image_path, distance=5):
    img = cv2.imread(image_path)

    # Check if the image is there
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return None  # Or handle the error appropriately

    img = cv2.resize(img, (128, 128))  # Resize for consistency

    # Use green channel (index 1 in BGR) instead of converting to grayscale
    green_channel = img[:, :, 1]  # BGR format in OpenCV, 1 is green

    # GLCM Texture Features using green channel
    glcm = graycomatrix(green_channel, distances=[distance], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

    # Color Histogram Features (RGB)
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]).flatten()

    # Edge Detection (Canny) using green channel
    edges = cv2.Canny(green_channel, 100, 200)
    edge_density = np.sum(edges) / (128 * 128)  # Ratio of edge pixels

    return np.hstack([contrast, correlation, energy, homogeneity, hist, edge_density])

# Test with different distance values
def test_distances(image_path):
    distances = [1, 3, 5, 7, 10]  # Different distance values to test
    results = {}

    for dist in distances:
        features = extract_features(image_path, distance=dist)
        if features is not None:
            results[dist] = {
                'contrast': features[0],
                'correlation': features[1],
                'energy': features[2],
                'homogeneity': features[3],
                'edge_density': features[-1]
            }
            print(f"Distance {dist}:")
            print(f"Contrast: {features[0]:.4f}")
            print(f"Correlation: {features[1]:.4f}")
            print(f"Energy: {features[2]:.4f}")
            print(f"Homogeneity: {features[3]:.4f}")
            print(f"Edge Density: {features[-1]:.4f}")
            print("--------------------")

    return results

In [ ]:
def load_reference_features(feature_csv):
    try:
        df = pd.read_csv(feature_csv, header=None)
        print("Loaded CSV shape:", df.shape)
        if df.empty:
            raise ValueError("CSV file is empty!")
        features = df.values.flatten()
        print("Loaded features length:", len(features))
        if len(features) != 517:
            raise ValueError(f"Expected 517 features, got {len(features)}")
        return features
    except Exception as e:
        print(f"Error loading reference features: {e}")
        return None

def classify_and_rename_images(image_folder, feature_csv, output_folder, threshold=0.8, distance=5):
    reference_features = load_reference_features(feature_csv)
    if reference_features is None:
        print("Aborting classification due to reference feature loading failure!")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            image_path = os.path.join(image_folder, filename)
            features = extract_features(image_path, distance=distance)
            if features is None:
                print(f"Skipping {filename} - feature extraction failed")
                continue

            try:
                similarity = np.dot(reference_features, features) / (
                    np.linalg.norm(reference_features) * np.linalg.norm(features)
                )
                label = "1" if similarity >= threshold else "0"
                new_filename = f"{label}_{filename}"
                new_path = os.path.join(output_folder, new_filename)
                img = cv2.imread(image_path)
                if img is not None:
                    cv2.imwrite(new_path, img)
                    print(f"Processed: {filename} → {new_filename} (Similarity: {similarity:.2f})")
                else:
                    print(f"Failed to read image: {filename}")
            except ValueError as e:
                print(f"Error computing similarity for {filename}: {e}")

# Paths
image_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/splitted_images"
feature_file = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/RGB_feature/extracted_RGB_feature.csv"
output_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/RGB_feature/labelled_images"

# Classify images
classify_and_rename_images(image_folder, feature_file, output_folder, threshold=0.8, distance=5)

# Sync Drive
drive.flush_and_unmount()

Loaded CSV shape: (1, 517)
Loaded features length: 517
Processed: tile_9.png → 1_tile_9.png (Similarity: 0.89)
Processed: tile_6.png → 1_tile_6.png (Similarity: 1.00)
Processed: tile_12.png → 1_tile_12.png (Similarity: 0.99)
Processed: tile_23.png → 0_tile_23.png (Similarity: 0.79)
Processed: tile_27.png → 1_tile_27.png (Similarity: 0.97)
Processed: tile_26.png → 1_tile_26.png (Similarity: 1.00)
Processed: tile_3.png → 1_tile_3.png (Similarity: 0.96)
Processed: tile_17.png → 1_tile_17.png (Similarity: 0.89)
Processed: tile_25.png → 1_tile_25.png (Similarity: 0.99)
Processed: tile_0.png → 0_tile_0.png (Similarity: 0.05)
Processed: tile_13.png → 1_tile_13.png (Similarity: 0.99)
Processed: tile_2.png → 1_tile_2.png (Similarity: 0.97)
Processed: tile_16.png → 1_tile_16.png (Similarity: 0.90)
Processed: tile_21.png → 0_tile_21.png (Similarity: 0.66)
Processed: tile_19.png → 1_tile_19.png (Similarity: 0.99)
Processed: tile_15.png → 0_tile_15.png (Similarity: 0.39)
Processed: tile_8.png → 0_t